In [1]:
import pandas as pd
import numpy as np 


In [23]:
dataset = pd.read_csv('../datasets/archive_en_p_heur.csv', sep=';')

In [24]:
dataset.head()

,id_device,date,en_L1,cons_en_L1,en_L2,cons_en_L2,en_L3,cons_en_L3,en_L4,cons_en_L4,...,min_THD_L4,max_THD_L4,moy_THD_L4,min_THD_L5,max_THD_L5,moy_THD_L5,min_THD_L6,max_THD_L6,moy_THD_L6,battery_level
0,34,16/07/2019 9:00,806.834961,806.834961,2069.452637,2069.452637,135.575562,135.575562,0,0,...,0,0,0,0,0,0,0,0,0,0
1,34,16/07/2019 10:00,2289.109131,1482.274170,5071.029297,3001.576660,1358.957397,1223.381836,0,0,...,0,0,0,0,0,0,0,0,0,0
2,34,16/07/2019 11:00,3494.928711,1205.819580,7951.330078,2880.300781,2303.930176,944.972778,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34,16/07/2019 12:00,4859.251465,1364.322754,10781.760742,2830.430664,3256.703369,952.773193,0,0,...,0,0,0,0,0,0,0,0,0,0
4,34,16/07/2019 13:00,6098.752930,1239.501465,13723.381836,2941.621094,4057.264160,800.560791,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
for k in dataset.keys():
    if len(dataset[k].value_counts()) <2:
        dataset.drop(columns=[k], inplace=True)

In [26]:
dataset['date'] = pd.to_datetime(dataset['date'])

In [27]:
data = dataset.copy()


In [28]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day_name()
data['hour'] = data['date'].dt.hour

In [31]:
data['cons_glbal'] = data['cons_en_L1'] + data['cons_en_L2'] + data['cons_en_L3'] 

In [36]:
months = dict(data["month"].value_counts())
months

{10: 6948,
 1: 6925,
 12: 6819,
 7: 6811,
 11: 6753,
 9: 6637,
 8: 6468,
 2: 6355,
 3: 6133,
 6: 4996,
 5: 4705,
 4: 4550}

In [37]:
def transform_data(data, number_of_samples, number_of_result):
    features = data['cons_glbal'].values.reshape(-1,1)
    X, Y = [], []
    list_months = list(data["month"])
    for month in months :
        start_index = list_months.index(month)
        for i in range (0,months[month]-number_of_samples,number_of_samples):
            X.append(features[start_index+i:start_index+i+number_of_samples,0])
            Y.append(features[start_index+i+number_of_samples:start_index+i+number_of_samples+number_of_result,0])

    return X, Y


In [39]:
X, Y = transform_data(data, 50, 1)

In [40]:
X = np.array(X)
Y = np.array(Y)

In [41]:
X.shape

(1476, 50)

In [43]:
Y.shape

(1476, 1)

In [50]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler2 = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
Y = scaler2.fit_transform(Y)

In [51]:
X_scaled = np.expand_dims(X_scaled, -1)
X_scaled.shape


(1476, 50, 1)

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size = 0.2, shuffle=True)

In [54]:
Y_train

array([[0.58107929],
       [0.57070444],
       [0.63197824],
       ...,
       [0.54568109],
       [0.50564403],
       [0.67442857]])

In [55]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [56]:
model = Sequential()
model.add(LSTM(50, input_shape=(50,1), activation="tanh"))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam')
model.fit(X_train, Y_train, validation_data=(X_test, Y_test),epochs=100, batch_size=56, verbose=True)

Epoch 1/100
22/22 [==============================] - 7s 126ms/step - loss: 0.1384 - val_loss: 0.0025
Epoch 2/100
22/22 [==============================] - 1s 34ms/step - loss: 0.0069 - val_loss: 0.0030
Epoch 3/100
22/22 [==============================] - 1s 31ms/step - loss: 0.0049 - val_loss: 0.0026
Epoch 4/100
22/22 [==============================] - 1s 35ms/step - loss: 0.0035 - val_loss: 0.0024
Epoch 5/100
22/22 [==============================] - 1s 32ms/step - loss: 0.0031 - val_loss: 0.0024
Epoch 6/100
22/22 [==============================] - 1s 30ms/step - loss: 0.0037 - val_loss: 0.0024
Epoch 7/100
22/22 [==============================] - 1s 33ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 8/100
22/22 [==============================] - 1s 29ms/step - loss: 0.0024 - val_loss: 0.0024
Epoch 9/100
22/22 [==============================] - 1s 31ms/step - loss: 0.0038 - val_loss: 0.0024
Epoch 10/100
22/22 [==============================] - 1s 33ms/step - loss: 0.0028 - val_loss: 0.002

22/22 [==============================] - 1s 30ms/step - loss: 0.0025 - val_loss: 0.0027
Epoch 83/100
22/22 [==============================] - 1s 31ms/step - loss: 0.0018 - val_loss: 0.0027
Epoch 84/100
22/22 [==============================] - 1s 31ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 85/100
22/22 [==============================] - 1s 30ms/step - loss: 0.0018 - val_loss: 0.0030
Epoch 86/100
22/22 [==============================] - 1s 39ms/step - loss: 0.0033 - val_loss: 0.0027
Epoch 87/100
22/22 [==============================] - 1s 33ms/step - loss: 0.0024 - val_loss: 0.0027
Epoch 88/100
22/22 [==============================] - 1s 30ms/step - loss: 0.0023 - val_loss: 0.0027
Epoch 89/100
22/22 [==============================] - 1s 33ms/step - loss: 0.0022 - val_loss: 0.0027
Epoch 90/100
22/22 [==============================] - 1s 34ms/step - loss: 0.0037 - val_loss: 0.0028
Epoch 91/100
22/22 [==============================] - 1s 35ms/step - loss: 0.0020 - val_loss: 0.0026
Epo

In [90]:
test = pd.read_csv('../datasets/archive_en_p_mois.csv', sep=';')

In [91]:
test['cons_glbal'] = test['cons_en_L1'] + test['cons_en_L2'] + test['cons_en_L3'] 

In [92]:
test['cons_glbal']

0      1.764375e+06
1      3.266888e+06
2      3.037124e+06
3      2.769838e+06
4      2.355590e+06
           ...     
101    1.373240e+07
102    1.408772e+07
103    1.263620e+07
104    1.160417e+07
105    9.934701e+06
Name: cons_glbal, Length: 106, dtype: float64

In [93]:
test = test['cons_glbal']
test

x_input = np.array(test[0:50])
input_MinMax = scaler.fit_transform(x_input.reshape(-1,1))
input_MinMax = input_MinMax.reshape(1,50,  -1)
x_itrainPredict = model.predict(input_MinMax)
prediction = scaler.inverse_transform(x_itrainPredict)
print('predc ',prediction)


predc  [[7146928.]]


In [97]:
score = model.evaluate(X_test, Y_test, verbose=1)

10/10 [==============================] - 0s 8ms/step - loss: 0.0027


In [99]:
score

0.0027429526671767235

In [103]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
coeff_determination